In [1]:
import glob
import tqdm
import os
import glob
import pandas as pd
import sys
import tarfile
import json
import shutil

from concurrent.futures import ThreadPoolExecutor, as_completed

sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath(__file__))))
from utils import audio_to_flac

ModuleNotFoundError: No module named 'tqdm'

In [2]:
emns_csv_path = '/home/knoriy/Documents/phd/dataset_collection_tool/django_dataset_collection_tool/audio_recorder_utterances.csv'
emns_audio_path = '/home/knoriy/Documents/phd/EMNS/wavs/'


In [1]:
raw_df = pd.read_csv(emns_csv_path, sep="|").dropna()

NameError: name 'pd' is not defined

# Create MFA Format

In [ ]:


def convert_and_json_dump(file:str, dest:str, df, overwrite:bool=False):
    if os.path.isfile(dest) and overwrite==False:
        print(f'{dest} already exists, skiping')
        return

    audio_to_flac(file, dest, segment_start=df['begin_time'], segment_end=df['end_time'])
    with open(dest.replace('.flac', '.txt'), 'w') as f:
        f.write(df['text'])
        # json.dump({'filename': os.path.join(*dest.split('/')[5:]), 'text':[df['text']], 'original_data':df['original_data']}, f)


def split_all_audio_files(df, dest_root_path, max_workers=96):
    if not os.path.exists(dest_root_path):
        raise FileNotFoundError(f'Please Check {dest_root_path} exists')

    l = len(df)
    with tqdm.tqdm(total=l, desc=f'Processing {dest_root_path}') as pbar:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            threads = [executor.submit(convert_and_json_dump, row[0], os.path.join(dest_root_path, f'{i}.flac'), row) for i, row in enumerate(df.iloc())]
            for _ in as_completed(threads):
                pbar.update(1)